In [112]:
import numpy as np

import torch
import torch.utils.data as data_utils

from pathlib import Path
from sklearn.model_selection import train_test_split

from data import Sequence,SequenceDataset
from model import LogNormMix

from copy import deepcopy
from util import pad_sequence
from torch.distributions import Categorical
from sklearn.metrics import f1_score

### Load Dataset

In [55]:
dataset_name = 'data/simulated/hawkes_synthetic_random_2d_20191130-180837.pkl'  # run dpp.data.list_datasets() to see the list of available datasets
dataset = np.load(dataset_name,allow_pickle = True)

## Modify the dataset for IFTPL




In [56]:
sequences = []

# for i  in range(len(dataset['timestamps'])):
for i  in range(400):
    sequence = {'t_start':0,'t_end' :200, 'arrival_times':dataset['timestamps'][i].tolist(),'marks' :dataset['types'][i].tolist()}
    
    sequences.append(sequence)
    
simulated_data = {'sequences':sequences,'num_marks':2}

# np.save('data/simulated/sample_hawkes',simulated_data,allow_pickle = True)

### Training

In [177]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)


# Model config
context_size = 64                 # Size of the RNN hidden vector
mark_embedding_size = 32          # Size of the mark embedding (used as RNN input)
num_mix_components = 64           # Number of components for a mixture model
rnn_type = "GRU"                  # What RNN to use as an encoder {"RNN", "GRU", "LSTM"}

# Training config
batch_size = 32        # Number of sequences in a batch
regularization = 1e-5  # L2 regularization parameter
learning_rate = 1e-3   # Learning rate for Adam optimizer
max_epochs = 5      # For how many epochs to train
display_step = 1      # Display training statistics after every display_step
patience = 50          # After how many consecutive epochs without improvement of val loss to stop training

test_dataset = {}
test_dataset['sequences']= [{'t_start': 0,'t_end': 10,'arrival_times': [1,2],'marks':[0,1]},
                            {'t_start': 0,'t_end': 10,'arrival_times': [1,3,5],'marks':[0,1,0]},
                            {'t_start': 0,'t_end': 10,'arrival_times': [1,7,8,9],'marks':[0,1,1,0]},
                            {'t_start': 0,   't_end': 10,'arrival_times': [5,9],'marks':[1,1]}  ]

def get_inter_times(seq: dict):
    """Get inter-event times from a sequence."""
    return np.ediff1d(np.concatenate([[seq["t_start"]], seq["arrival_times"], [seq["t_end"]]]))

sequences = [
    Sequence(
        inter_times=get_inter_times(seq),
        marks=seq.get("marks"),
        t_start=seq.get("t_start"),
        t_end=seq.get("t_end")
    )
    for seq in simulated_data["sequences"]
]

seed = 0
batch_size = 5
dataset = SequenceDataset(sequences=sequences, num_marks=2)

d_train, d_val, d_test = dataset.train_val_test_split(seed=None,shuffle = False)

training_events= d_train.total_num_events

dl_train = d_train.get_dataloader(batch_size=batch_size, shuffle=False)
dl_val = d_val.get_dataloader(batch_size=batch_size, shuffle=False)
dl_test = d_test.get_dataloader(batch_size=batch_size, shuffle=False)

# dataset_name = 'data/stack_overflow.pkl'  # run dpp.data.list_datasets() to see the list of available datasets
# dataset = torch.load(dataset_name)


# def get_inter_times(seq: dict):
#     """Get inter-event times from a sequence."""
#     return np.ediff1d(np.concatenate([[seq["t_start"]], seq["arrival_times"], [seq["t_end"]]]))


# sequences = [
#     Sequence(
#         inter_times=get_inter_times(seq),
#         marks=seq.get("marks"),
#         t_start=seq.get("t_start"),
#         t_end=seq.get("t_end")
#     )
#     for seq in dataset["sequences"]
# ]
# dataset = SequenceDataset(sequences=sequences, num_marks=dataset.get("num_marks", 1))

# d_train, d_val, d_test = dataset.train_val_test_split(seed=seed)

# dl_train = d_train.get_dataloader(batch_size=batch_size, shuffle=True)
# dl_val = d_val.get_dataloader(batch_size=batch_size, shuffle=False)
# dl_test = d_test.get_dataloader(batch_size=batch_size, shuffle=False)





In [195]:
batches = []
for batch in dl_test:
    batches.append(batch)
batches[0]

Batch(inter_times=[5, 176], marks=[5, 176], mask=[5, 176])

In [193]:
len(dl_test.dataset.sequences[0].inter_times)

128

In [178]:

# Define the model
print('Building model...')
mean_log_inter_time, std_log_inter_time = d_train.get_inter_time_statistics()

model =LogNormMix(
    num_marks=d_train.num_marks,
    mean_log_inter_time=mean_log_inter_time,
    std_log_inter_time=std_log_inter_time,
    context_size=context_size,
    mark_embedding_size=mark_embedding_size,
    rnn_type=rnn_type,
    num_mix_components=num_mix_components,
)

opt = torch.optim.Adam(model.parameters(), weight_decay=regularization, lr=learning_rate)
# Traning
print('Starting training...')

def aggregate_loss_over_dataloader(dl):
    total_loss = 0.0
    total_count = 0
    with torch.no_grad():
        for batch in dl:
            total_loss += -model.log_prob(batch).sum()
            total_count += batch.mask.sum().item()
    return total_loss / total_count


impatient = 0
best_loss = np.inf
best_model = deepcopy(model.state_dict())
training_val_losses = []

Building model...
Starting training...


In [179]:
for epoch in range(max_epochs):
    epoch_train_loss = 0
    model.train()
    for batch in dl_train:
        opt.zero_grad()
        # loss = -model.log_prob(batch)
        loss = -model.log_prob(batch).sum()
        loss.backward()
        epoch_train_loss += loss.detach()

        opt.step()

    model.eval()
    with torch.no_grad():
        loss_val = aggregate_loss_over_dataloader(dl_val)
        loss_test = aggregate_loss_over_dataloader(dl_test)

        training_val_losses.append(loss_val)

    if (best_loss - loss_val) < 1e-4:
        impatient += 1
        if loss_val < best_loss:
            best_loss = loss_val
            best_model = deepcopy(model.state_dict())
    else:
        best_loss = loss_val
        best_model = deepcopy(model.state_dict())
        impatient = 0

    if impatient >= patience:
        print(f'Breaking due to early stopping at epoch {epoch}')
        break
    
    
    epoch_train_loss = epoch_train_loss/training_events

    if epoch % display_step == 0:
        print(f"Epoch {epoch:4d}: Training loss = {epoch_train_loss.item():.4f}, loss_val = {loss_val:.4f}")


# Evaluation
model.load_state_dict(best_model)
model.eval()

# All training & testing sequences stacked into a single batch
with torch.no_grad():
    final_loss_train = aggregate_loss_over_dataloader(dl_train)
    final_loss_val = aggregate_loss_over_dataloader(dl_val)
    final_loss_test = aggregate_loss_over_dataloader(dl_test)

print(f'Negative log-likelihood:\n'
      f' - Train: {final_loss_train:.4f}\n'
      f' - Val:   {final_loss_val:.4f}\n'
      f' - Test:  {final_loss_test:.4f}')




Epoch    0: Training loss = 1.8333, loss_val = 1.8100
Epoch    1: Training loss = 1.7969, loss_val = 1.8056
Epoch    2: Training loss = 1.7926, loss_val = 1.8009
Epoch    3: Training loss = 1.7895, loss_val = 1.7984
Epoch    4: Training loss = 1.7871, loss_val = 1.7972
Negative log-likelihood:
 - Train: 1.8
 - Val:   1.8
 - Test:  1.7


### Event Prediction 

In [201]:
def get_prediction_for_all_events(model,dl):
    
    total_num_events = dl.dataset.total_num_events
    all_event_time_predictions = []
    all_event_time_values = []
    all_actual_marks = []
    all_predicted_marks = []
    
    
    for batch in dl:
        
        lengths = batch.mask.sum(-1)-1 ## Minus 1 because they also calculate the survival for the end of time.

        
        features = model.get_features(batch)
        context = model.get_context(features)
        inter_time_dist = model.get_inter_time_dist(context)
        inter_times = batch.inter_times.clamp(1e-10)
        
        ## Arrival Time Prediction
        predicted_times = inter_time_dist.mean        
        all_predicted_times = torch.nn.utils.rnn.pack_padded_sequence(predicted_times.T,lengths,batch_first=False,enforce_sorted=False)[0]
        all_actual_times = torch.nn.utils.rnn.pack_padded_sequence(inter_times.T,lengths,batch_first=False,enforce_sorted=False)[0]
        all_event_time_values.append(all_actual_times[:-1])
        all_event_time_predictions.append(all_predicted_times[:-1])
        


#         ## Mark Prediction
        predicted_marks= torch.log_softmax(model.mark_linear(context), dim=-1).argmax(-1)
        predicted_marks = torch.nn.utils.rnn.pack_padded_sequence(predicted_marks.T,lengths,batch_first=False,enforce_sorted=False)[0]
        actual_marks = torch.nn.utils.rnn.pack_padded_sequence(batch.marks.T,lengths,batch_first=False,enforce_sorted =False)[0]
        all_actual_marks.append(actual_marks)
        all_predicted_marks.append(predicted_marks)
        
#         all_event_accuracy = (predicted_marks ==batch.marks)*batch.mask
#         all_event_accuracies.append(all_event_accuracy)
#         last_event_accuracy = all_event_accuracy[x_index,y_index]
#         last_event_accuracies.append(last_event_accuracy)
        
    
#     last_event_rmse = (torch.cat(last_event_errors,axis = 0)**2).mean().sqrt()
#     all_event_rmse = ((torch.cat(total_errors,-1)**2).sum()/total_num_events).sqrt()
#     all_event_accuracy = torch.cat(all_event_accuracies,-1).sum()/total_num_events
#     last_event_accuracy = torch.cat(last_event_accuracies,0)
# #     last_event_accuracy = last_event_accuracy.sum()/len(last_event_accuracy)
    
#     return last_event_rmse,all_event_rmse,all_event_accuracy,last_event_accuracy
    
    
    
    all_event_time_values = torch.cat(all_event_time_values)
    all_event_time_predictions = torch.cat(all_event_time_predictions)
    all_actual_marks = torch.cat(all_actual_marks)
    all_predicted_marks = torch.cat(all_predicted_marks)
    
    return all_event_time_values,all_event_time_predictions,all_actual_marks,all_predicted_marks

In [202]:
def get_prediction_for_last_events(model,dl):
    
    event_time_predictions = []
    event_time_values = []
    actual_marks = []
    predicted_marks = []
    
    
    for batch in dl:
        
        y_index = batch.mask.sum(-1).long() -1  ## Minus 1 because they also calculate the survival for the end of time.
        features = model.get_features(batch)
        context = model.get_context(features)
        inter_time_dist = model.get_inter_time_dist(context)
        inter_times = batch.inter_times.clamp(1e-10)
        x_index = torch.arange(0,len(inter_times))
        
        ## Arrival Time Prediction
        actual_time = inter_times[x_index,y_index]   
        predicted_time = inter_time_dist.mean[x_index,y_index]     

        ## Mark Prediction
        actual_mark = batch.marks[x_index,y_index]
        predicted_mark= torch.log_softmax(model.mark_linear(context), dim=-1).argmax(-1)[x_index,y_index]
        
        event_time_predictions.append(predicted_time)
        event_time_values.append(actual_time)
        actual_marks.append(actual_mark)
        predicted_marks.append(predicted_mark)
    
    event_time_values = torch.cat(event_time_values)
    event_time_predictions = torch.cat(event_time_predictions)
    actual_marks = torch.cat(actual_marks)
    predicted_marks = torch.cat(predicted_marks)
    
    return event_time_values,event_time_predictions,actual_marks,predicted_marks

In [208]:
batch.marks.device

device(type='cpu')

In [203]:
actual_times,predicted_times,actual_marks,predicted_marks = get_prediction_for_all_events(model,dl_test)

RMSE = (((predicted_times - actual_times)/actual_times)**2).mean().sqrt()
f1= f1_score(predicted_marks.detach().numpy(),actual_marks.detach().numpy())

print(RMSE.item())
print(f1)

218.84422302246094
0.550259965337955


In [204]:
actual_times,predicted_times,actual_marks,predicted_marks = get_prediction_for_last_events(model,dl_test)

RMSE = (((predicted_times - actual_times)/actual_times)**2).mean().sqrt()
f1= f1_score(predicted_marks.detach().numpy(),actual_marks.detach().numpy())

print(RMSE.item())
print(f1)

13.017912864685059
0.4571428571428572


In [217]:
batch.marks.cpu()

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]])